In [1]:
import pandas as pd  # type: ignore
import numpy as np  # type: ignore

import os
for dirname, _, filenames in os.walk('/Users/tavsiev/Desktop/projet Rynk-Kaggle-24:25/classification-multi-classes'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.model_selection import train_test_split # type 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, classification_report

/Users/tavsiev/Desktop/projet Rynk-Kaggle-24:25/classification-multi-classes/test.csv
/Users/tavsiev/Desktop/projet Rynk-Kaggle-24:25/classification-multi-classes/train.csv
/Users/tavsiev/Desktop/projet Rynk-Kaggle-24:25/classification-multi-classes/sample_submission.csv


In [2]:
train_df = pd.read_csv("/Users/tavsiev/Desktop/projet Rynk-Kaggle-24:25/classification-multi-classes/train.csv", 
                      delimiter = ",",
                      header = 0)

train_df.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,1055.0,NaN,19724.0,F,NaN,NaN,NaN,N,1.3,NaN,3.64,NaN,NaN,NaN,NaN,209.0,10.5,3.0,C
1,1,3282.0,Placebo,17884.0,F,N,Y,Y,N,0.7,309.0,3.60,96.0,1142.0,71.3,106.0,240.0,12.4,4.0,C
2,2,1653.0,NaN,20600.0,F,NaN,NaN,NaN,N,2.2,NaN,3.64,NaN,NaN,NaN,NaN,139.0,9.5,2.0,C
3,3,999.0,D-penicillamine,22514.0,F,N,Y,N,N,1.0,498.0,3.35,89.0,1601.0,164.3,85.0,394.0,9.7,3.0,C
4,4,2202.0,NaN,17897.0,F,NaN,NaN,NaN,N,17.2,NaN,3.15,NaN,NaN,NaN,NaN,432.0,11.2,3.0,C


In [3]:
train_df.shape
# total de 15 000 lignes
# priorité: traitement des NA

(15000, 20)

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15000 non-null  int64  
 1   N_Days         15000 non-null  float64
 2   Drug           8564 non-null   object 
 3   Age            15000 non-null  float64
 4   Sex            15000 non-null  object 
 5   Ascites        8563 non-null   object 
 6   Hepatomegaly   8564 non-null   object 
 7   Spiders        8555 non-null   object 
 8   Edema          15000 non-null  object 
 9   Bilirubin      15000 non-null  float64
 10  Cholesterol    6669 non-null   float64
 11  Albumin        15000 non-null  float64
 12  Copper         8452 non-null   float64
 13  Alk_Phos       8558 non-null   float64
 14  SGOT           8555 non-null   float64
 15  Tryglicerides  6630 non-null   float64
 16  Platelets      14414 non-null  float64
 17  Prothrombin    14974 non-null  float64
 18  Stage 

In [5]:
train_df['Status'].value_counts()
# les classes sont déséquilibrées 
# en tenir compte 

Status
C     10049
D      4560
CL      391
Name: count, dtype: int64

Status: C si le patient est encore vivant

Status: CL si le patient est vivant mais transplanté

Status: D si le patient est mort


In [6]:
# problème au niveau de l'ID? 
# train_df['id'].value_counts().sort_values().sum()

# ID en int, on transforme en object
# train_df['id'] = train_df['id'].astype('object')

# len(train_df['id'].unique())
# train_df['id'].value_counts().sort_values()

À priori un unique ID est attribué à chaque patient.

In [7]:
# train_df.info()
# ID OK 
# AU NIVEAU DES AUTRES COLONNES?
# POUR: id, N_Days, Age, Sex, Edema, Bilirubin, Albumin, Stage, Status PAS DE NA
# DONC OK

In [8]:
train_df['Drug'].value_counts()

Drug
D-penicillamine    4568
Placebo            3996
Name: count, dtype: int64

In [9]:
# parfois on a des cases vides, ou des cases avec 'NA', 'None', 'nan' mais Python les détecte 
# pas toujours de la même façon
# je fais la somme des NA et des colonnes non nulles pour m'en assurer
print(train_df['Ascites'].value_counts())
print(train_df['Ascites'].isna().sum())
print(train_df['Ascites'].notnull().sum())
print(train_df['Ascites'].isna().sum() + train_df['Ascites'].notnull().sum())

# boucle en bas pour toutes les variables

Ascites
N                  8178
Y                   383
S                     1
D-penicillamine       1
Name: count, dtype: int64
6437
8563
15000


In [10]:
# for col in train_df.columns:
#    print(f"--- Analyse de la colonne: {col} ---")
#    print("Valeurs uniques et leurs occurrences:")
#    print(train_df[col].value_counts(dropna=False))  # Inclut NaN dans le comptage
#    print(f"Valeurs manquantes (NaN): {train_df[col].isna().sum()}")
#    print(f"Valeurs non manquantes: {train_df[col].notnull().sum()}")
#    total = train_df[col].isna().sum() + train_df[col].notnull().sum()
#    print(f"Total (manquantes + non manquantes): {total}")
#    print()

# OK, VERIFIÉ

Regarder le taux de NA pour chaque variable.

Il est possible que ce soit les mêmes individus pour lesquels on a des NA, donc un groupe qui n'aurait pas suivi le protocole.

Voir la variable 'Drug' où y a eu un placebo et un médicament testé mais cela représente que 8500 personnes. Or, environ 6500 n'ont rien eu donc soit c'est placebo, soit ils n'ont pas participé dutout à l'expérience.

In [11]:
# les valeurs que prennent les colonnes de type 'object'
for col in train_df.select_dtypes(include=['object']).columns:
    unique_values = ', '.join(map(str, train_df[col].unique()))
    print(f"Colonne '{col}': {unique_values}")


# N --> No 
# Y --> Yes
# F --> Female
# M --> Male

# pour 'Drug': Placebo et D-penicillamine
# mais pour 'Ascites', la valeur D-penicillamine est une erreur 
# mais pour 'Ascites', la valeur de S est une erreur 
# 'Ascites' prend que N (oui) et Y (non) 
# dans Edema, le S apparaît 800 fois donc OK 

Colonne 'Drug': nan, Placebo, D-penicillamine
Colonne 'Sex': F, M
Colonne 'Ascites': nan, N, Y, S, D-penicillamine
Colonne 'Hepatomegaly': nan, Y, N, 119.35
Colonne 'Spiders': nan, Y, N
Colonne 'Edema': N, Y, S
Colonne 'Status': C, D, CL


In [12]:
train_df = train_df[~train_df['Ascites'].isin(['D-penicillamine', 'S'])]
train_df = train_df[train_df['Hepatomegaly'] != '119.35']

train_df.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,1055.0,NaN,19724.0,F,NaN,NaN,NaN,N,1.3,NaN,3.64,NaN,NaN,NaN,NaN,209.0,10.5,3.0,C
1,1,3282.0,Placebo,17884.0,F,N,Y,Y,N,0.7,309.0,3.60,96.0,1142.0,71.3,106.0,240.0,12.4,4.0,C
2,2,1653.0,NaN,20600.0,F,NaN,NaN,NaN,N,2.2,NaN,3.64,NaN,NaN,NaN,NaN,139.0,9.5,2.0,C
3,3,999.0,D-penicillamine,22514.0,F,N,Y,N,N,1.0,498.0,3.35,89.0,1601.0,164.3,85.0,394.0,9.7,3.0,C
4,4,2202.0,NaN,17897.0,F,NaN,NaN,NaN,N,17.2,NaN,3.15,NaN,NaN,NaN,NaN,432.0,11.2,3.0,C


In [13]:
for col in train_df.select_dtypes(include=['object']).columns:
    unique_values = ', '.join(map(str, train_df[col].unique()))
    print(f"Colonne '{col}': {unique_values}")

Colonne 'Drug': nan, Placebo, D-penicillamine
Colonne 'Sex': F, M
Colonne 'Ascites': nan, N, Y
Colonne 'Hepatomegaly': nan, Y, N
Colonne 'Spiders': nan, Y, N
Colonne 'Edema': N, Y, S
Colonne 'Status': C, D, CL


In [14]:
# format des colonnes + âge (en Jours vers Années)
train_df['Age'] = (train_df['Age'] / 365.25).round()

cat_col = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

# One-Hot Encodaging pour les colonnes catégorielles
train_df_encoded = pd.get_dummies(train_df, columns=cat_col, drop_first=True)

### **Test rapide :**

In [15]:
train_df_clean = train_df_encoded.dropna()
train_df_clean.shape

(6336, 21)

In [16]:
train_df_clean['Status'] = train_df_clean['Status'].map({'C': 0, 'CL': 1, 'D': 2})

/var/folders/zc/__4fgnp121vflx6jgwz_rg240000gn/T/ipykernel_2345/3080838254.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_clean['Status'] = train_df_clean['Status'].map({'C': 0, 'CL': 1, 'D': 2})


In [17]:
X = train_df_clean.drop(['id', 'Status'], axis=1) 
y = train_df_clean['Status']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=10000)

model.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=10000, multi_class='multinomial')

In [19]:
y_pred_proba = model.predict_proba(X_test)

logloss = log_loss(y_test, y_pred_proba)
print(f"Log Loss: {logloss}")

Log Loss: 0.4316062473968072


In [20]:
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred, target_names=['C', 'CL', 'D']))

              precision    recall  f1-score   support

           C       0.87      0.94      0.90       873
          CL       1.00      0.04      0.08        46
           D       0.78      0.72      0.75       349

    accuracy                           0.85      1268
   macro avg       0.88      0.57      0.58      1268
weighted avg       0.85      0.85      0.83      1268

